In [1]:
from my_weapon import *
from SQLite_handler import *
from tqdm.notebook import tqdm as tqdm

plt.style.use('fivethirtyeight')

# Rural and Urban

In [2]:
rural_table = pd.read_csv("data/ruralurbancodes2013.csv")
rural_table["Population"] = rural_table["Population_2010"].apply(lambda x: int(x.replace(",", "")))
rural_table["is_rural"] = rural_table["RUCC_2013"].apply(lambda x: 1 if x >= 3 else 0) # 分类，大于3以上是大城市
# rural_table[rural_table.County_Name=="Orleans Parish"]

# def county_to_rural(state_name, county_name):
#     try:
#         return rural_table[(rural_table["State"]==state_name) & (rural_table["county"]==county_name)]["is_rural"].iloc[0]
#     except:
#         return f"cannot find the loc ({state_name}, {county_name})"

from collections import defaultdict
dict_state_county_is_rural = defaultdict(dict)

for row in rural_table.itertuples():
    State = getattr(row, "State")
    county = getattr(row, "County_Name")
    is_rural = getattr(row, "is_rural")
    if county != "District of Columbia":
        county = " ".join(county.split()[: -1])
    dict_state_county_is_rural[State][county] = is_rural
    
# print(dict_state_county_is_rural["LA"])

rural_population = rural_table.groupby(["State", "is_rural"]).sum()["Population"]
rural_count = dict(rural_population)

In [3]:
rural_table

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description,Population,is_rural
0,1001,AL,Autauga County,"54,571",2,"Metro - Counties in metro areas of 250,000 to ...",54571,0
1,1003,AL,Baldwin County,"182,265",3,Metro - Counties in metro areas of fewer than ...,182265,1
2,1005,AL,Barbour County,"27,457",6,"Nonmetro - Urban population of 2,500 to 19,999...",27457,1
3,1007,AL,Bibb County,"22,915",1,Metro - Counties in metro areas of 1 million p...,22915,0
4,1009,AL,Blount County,"57,322",1,Metro - Counties in metro areas of 1 million p...,57322,0
...,...,...,...,...,...,...,...,...
3216,72145,PR,Vega Baja Municipio,"59,662",1,Metro - Counties in metro areas of 1 million p...,59662,0
3217,72147,PR,Vieques Municipio,"9,301",7,"Nonmetro - Urban population of 2,500 to 19,999...",9301,1
3218,72149,PR,Villalba Municipio,"26,073",2,"Metro - Counties in metro areas of 250,000 to ...",26073,0
3219,72151,PR,Yabucoa Municipio,"37,941",1,Metro - Counties in metro areas of 1 million p...,37941,0


## Load the users' opinions and locations

In [8]:
users_loc = pd.read_csv("data/202006-all-users.csv", usecols=["uid", "state", "county"], index_col="uid")
users_loc = users_loc.dropna()
users_loc["county"] = users_loc["county"].apply(lambda x: " ".join(x.split(", ")[:-1]))
users_loc

,county,state
uid,,
140299630,Berkeley,CA
984471367176241152,Washington,DC
6497512,Hoboken,NJ
2808545932,Detroit,MI
357494668,Middletown,CT
...,...,...
1077379415867314176,Cedar Hill,TX
1084938692265820160,New York,NY
264400122,New York,NY


In [9]:
USA_STATES = ['NY', 'DC', 'IN', 'AR', 'WY', 'ME', 'TX', 'NH', 'CO', 'CA', 'IL',
             'WA', 'VA', 'FL', 'MA', 'OR', 'AZ', 'MT', 'MN', 'NE', 'TN', 'OH',
             'NJ', 'NV', 'KY', 'UT', 'NC', 'SC', 'PA', 'NM', 'KS', 'GA', 'MI',
             'WI', 'AK', 'MS', 'MD', 'LA', 'HI', 'MO', 'AL', 'CT', 'OK', 'IA',
             'WV', 'RI', 'SD', 'VT', 'ND', 'ID', 'DE']

In [10]:
# Since we only know the name of city, but the county. We need convert the city to county.
# check some mapping from http://www.statsamerica.org/CityCountyFinder/Default.aspx

_dict_city_to_county = {
    'Chicago, IL': 'Cook',
    'Washington, DC': 'District of Columbia',
    'Atlanta, GA': 'Fulton',
    'Boston, MA': 'Suffolk',
    'Seattle, WA': 'King',
    'Brooklyn, NY': 'New York',
    'Miami, FL': 'Miami-Dade',
    'Las Vegas, NV': 'Clark',
    'Portland, OR': 'Multnomah',
    'Pittsburgh, PA': 'Allegheny',
    'San Antonio, TX': 'Bexar',
    'Nashville, TN': 'Davidson',
    'Orlando, FL': 'Orange',
    'Phoenix, AZ': 'Maricopa',
    'Indianapolis, IN': 'Marion',
    'Columbus, OH': 'Franklin',
    'Minneapolis, MN': 'Hennepin',
    'St Louis, MO': 'St. Louis',
    'Charlotte, NC': 'Mecklenburg',
    'Cleveland, OH': 'Cuyahoga',
    'Detroit, MI': 'Wayne',
    'Tampa, FL': 'Hillsborough',
    'New Orleans, LA': 'Orleans',
    'Kansas City, MO': 'Jackson',
    'Cincinnati, OH': 'Hamilton'
}
    
def city_to_county(state_name, city_name):
    return _dict_city_to_county.get(f"{city_name}, {state_name}", city_name)

In [12]:
from collections import Counter

find_county = Counter()
cannot_find_county = Counter()

for _state in tqdm(USA_STATES):
    _users = users_loc[users_loc.state==_state].copy()
    _users["county"] = _users["county"].apply(lambda x: city_to_county(_state, x))
    _users["is_rural"] = _users["county"].apply(lambda x: dict_state_county_is_rural[_state].get(x, -1))
    _users.to_csv(f"data/is_rural/{_state}.csv")

#     for row in _users.itertuples():
#         county = getattr(row, "county")
#         is_rural = getattr(row, "is_rural")
#         if is_rural == -1:
#             cannot_find_county[f"{county}, {_state}"] += 1
#         else:
#             find_county[f"{county}, {_state}"] += 1
    
    
# -1 represents that we cannot map the city (from BingMap) to county
# So we need more infomation between city and county

In [13]:
print(len(cannot_find_county))
print(cannot_find_county.most_common(10))
find_county.most_common(10)

0

In [22]:
# load users' opinion
# users_opinion = pd.read_csv("data/tweetid_userid_pro_hillary.csv")
import pendulum
users_opinion = defaultdict(lambda: [0, 0])
# set_users = set(users_loc.index)
# start = pendulum.datetime(2016, 9, 1)

# election date: 2016-11-8
# for line in tqdm(open("data/tweetid_userid_pro_hillary.csv")):
#     w = line.strip().split(",")
#     uid = int(w[0])
#     if uid in set_users:
#         if float(w[2]) < 0.5:
#             users_opinion[uid][0] += 1
#         else:
#             users_opinion[uid][1] += 1

users_opinion_v2 = []
# election date: 2016-11-8
i = 0
for line in tqdm(open("data/2020-07-01.csv")):
    w = line.strip().split(",")
    i += 1
    if i == 1:
        continue
    uid = int(w[0])
    users_opinion_v2.append({
        "uid": uid,
        "Camp": "T" if int(w[2]) > int(w[1]) else "B" 
    })
users_opinion = pd.DataFrame(users_opinion_v2).set_index("uid")
users_opinion


,Camp
uid,
314551800,T
1080662158415822849,B
3407179563,B
823998756655861760,B
79236840,T
...,...
14990801,T
4846877070,T
3182944844,T


In [24]:
for _state in tqdm(USA_STATES):
    users_county = pd.read_csv("data/is_rural/" + _state + ".csv", index_col="uid")
    users_county = users_county.join(users_opinion)
    groups = dict(users_county.groupby(["is_rural", "Camp"]).size())
    # print(_state, groups) 

    w1 = rural_count.get((_state, 1), 0) / (rural_count.get((_state, 1), 0) + rural_count.get((_state, 0), 0))
    w0 = 1 - w1
    
    groups_T_H = dict(users_county[users_county.is_rural != -1].groupby("Camp").size())
    # print(groups_T_H)
    proT = groups_T_H.get("T", 0) / (groups_T_H.get("T", 0) + groups_T_H.get("B", 0))
    

    if (0, 'T') not in groups: # No urban people support Trump!
        pro_T_1 = groups[(1, 'T')] / (groups[(1, 'T')] + groups[(1, 'B')])
        rst = pro_T_1
    elif (1, 'T') not in groups: # No rural people support Trump!
        pro_T_0 = groups[(0, 'T')] / (groups[(0, 'T')] + groups[(0, 'B')])
        rst = pro_T_0
    else:
        pro_T_0 = groups[(0, 'T')] / (groups[(0, 'T')] + groups[(0, 'B')])
        pro_T_1 = groups[(1, 'T')] / (groups[(1, 'T')] + groups[(1, 'B')])
        rst = pro_T_1 * w1 + pro_T_0 * w0
        
    print(f"{_state}, Census (rural areas): {w1 * 100:.2f}%, Trump: {proT * 100:.2f}%, After rescaling: {rst * 100:.2f}%")

NY {(-1, 'B'): 45947, (-1, 'T'): 27660, (0, 'B'): 182700, (0, 'T'): 82911, (1, 'B'): 208, (1, 'T'): 97}
NY, Census (rural areas): 10.53%, Trump: 31.22%, After rescaling: 31.28%
DC {(-1, 'B'): 88, (-1, 'T'): 35, (0, 'B'): 65186, (0, 'T'): 29879}
DC, Census (rural areas): 0.00%, Trump: 31.43%, After rescaling: 31.43%
IN {(-1, 'B'): 16828, (-1, 'T'): 11934, (0, 'B'): 11912, (0, 'T'): 7951, (1, 'B'): 476, (1, 'T'): 416}
IN, Census (rural areas): 40.09%, Trump: 40.31%, After rescaling: 42.68%


KeyboardInterrupt: 